**Here we will answer questions about our data**

Import our dataframes 

utilize pandasql (sqldf) in order to make these "sqlable" 

write queries that answer said questions

In [76]:
#Imports 
from pandasql import sqldf
import pandas as pd

RewardsReceiptItems = pd.read_csv('clean_data/receipts_items.csv', index_col = 0)
Receipts = pd.read_csv('clean_data/receipts.csv', index_col = 0)
Brands = pd.read_csv('clean_data/brands.csv', index_col = 0)
Users = pd.read_csv('clean_data/users.csv', index_col = 0)

In [9]:
#check the data
#note there is a lot of duplicate user ids
Users

,active,createdDate,lastLogin,role,signUpSource,state,userId
0,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
1,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
2,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
3,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6
4,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...
490,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN,54943462e4b07e684157a532
491,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN,54943462e4b07e684157a532
492,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN,54943462e4b07e684157a532
493,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN,54943462e4b07e684157a532


In [11]:
pysqldf = lambda q: sqldf(q, globals())

Question 1: What are the top 5 brands by receipts scanned for most recent month?

In [ ]:
#quick check to see what the most recent month is - since we determined that its
#the first of the month 2021-03, we will filter for any values >= 02-01 and filter out 
#values from 03-01
query = """SELECT max(dateScanned)

from Receipts"""

In [64]:
query = """
SELECT  count(RewardsReceiptItems.brandCode) as receipt_count,

        RewardsReceiptItems.brandCode
        
        FROM RewardsReceiptItems INNER JOIN Receipts
        
        ON RewardsReceiptItems.receiptId = Receipts.receiptId 
        
        WHERE dateScanned >= '2021-02-01' and dateScanned < '2021-03-01'
        
        and  brandCode is not Null
        
        GROUP BY brandCode ORDER BY receipt_count DESC limit 5;"""

In [65]:
pysqldf(query)

,receipt_count,brandCode
0,3,BRAND
1,2,MISSION
2,1,VIVA


In [55]:
#even though this is not helpful, it is what the question asked.
#most events seem to be in January, in which case the answer is below. 

In [60]:
query = """
SELECT  count(RewardsReceiptItems.receiptId) as receipt_item_count,

        RewardsReceiptItems.brandCode
        
        FROM RewardsReceiptItems INNER JOIN Receipts
        
        ON RewardsReceiptItems.receiptId = Receipts.receiptId 
        
        WHERE dateScanned >= '2021-01-01' and dateScanned < '2021-02-01'
        
        and  brandCode is not Null
        
        GROUP BY brandCode ORDER BY receipt_item_count DESC limit 5;"""

In [61]:
pysqldf(query)

,receipt_item_count,brandCode
0,291,HY-VEE
1,180,BEN AND JERRYS
2,93,PEPSI
3,89,KROGER
4,88,KLEENEX


Question 2:  

Given the above queries, it is easy to see we may be having data quality issues in tracking brand.  The receipt item count from January is completely different than Feb where only 6 events were valid. 

In [46]:
query = """
SELECT Receipts.receiptId, count(RewardsReceiptItems.receiptId) as receipt_item_count,

        RewardsReceiptItems.brandCode
        
        FROM RewardsReceiptItems INNER JOIN Receipts
        
        ON RewardsReceiptItems.receiptId = Receipts.receiptId 
        
        WHERE dateScanned >= '2020-12-01' and dateScanned < '2021-01-01'
        
        and  brandCode is not Null
        
        GROUP BY brandCode ORDER BY receipt_item_count DESC limit 5;"""

In [47]:
pysqldf(query)

,receiptId,receipt_count,brandCode


Question 3:
When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Note: I am going to assume we mean finished or rejected, because I checked this data as well as raw and there were no accepted statuses

In [74]:
query = """SELECT Receipts.rewardsReceiptStatus as rewardsReceiptStatus, 
        
        avg(Receipts.totalSpent) as average_spend

        FROM RewardsReceiptItems INNER JOIN Receipts
        
        ON RewardsReceiptItems.receiptId = Receipts.receiptId

        where rewardsReceiptStatus IN ('ACCEPTED', 'FINISHED', 'REJECTED')
        
        GROUP BY rewardsReceiptStatus;"""

In [75]:
pysqldf(query)

,rewardsReceiptStatus,average_spend
0,FINISHED,1244.372934
1,REJECTED,19.544970


It can be seen that average spend is much higher in the finished state!

Question 4:
When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Note: I am going to assume we mean finished or rejected, because I checked this data as well as raw and there were no accepted statuses

In [77]:
query = """SELECT Receipts.rewardsReceiptStatus as rewardsReceiptStatus, 
        
        count(Receipts.purchasedItemCount) as purchasedItemCount

        FROM RewardsReceiptItems INNER JOIN Receipts
        
        ON RewardsReceiptItems.receiptId = Receipts.receiptId

        where rewardsReceiptStatus IN ('ACCEPTED', 'FINISHED', 'REJECTED')
        
        GROUP BY rewardsReceiptStatus;"""

In [78]:
pysqldf(query)

,rewardsReceiptStatus,purchasedItemCount
0,FINISHED,5920
1,REJECTED,167


It can be seen that items purchased is much higher in the finished state!